In [1]:
from datetime import datetime

import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact
import folium
import re
import random

In [9]:
#Загрузка данных для разработки датасета
df = pd.read_excel(io='data.xlsx', index_col=0)

# Взять только последние лицензии для исключения дубликатов
# Обработать данные 
geodata = df.loc[df['Last'] == True]

In [4]:
#Функция для извлечения радиан из столбцов N и E
def to_rads(value) -> float:

    pattern = "(\d*)°(\d*)'(\d*\.*\d*)"
    val = re.findall(pattern=pattern, string=value)[0]
    return round(float(val[0]) + float(val[1])/60 + float(val[2])/3600, 5)

In [20]:
#Добавление столбцов
#Перевод координат град-мин-сек в радианы
geodata.loc[:,'rad_N'] = geodata['N'].map(arg=to_rads, na_action='ignore')
geodata.loc[:,'rad_E'] = geodata['E'].map(arg=to_rads, na_action='ignore')

#Добавление периодов
geodata.loc[:,'month'] = geodata['date'].dt.to_period('M')

In [22]:
geodata.to_excel(excel_writer='geodata.xlsx', engine='openpyxl')

In [23]:
#Доработать

In [ ]:
def coords(period):
    if period:
        start_date = period[0]
        end_date = period[1]
        mask = (geodata['month'] >= start_date) & (geodata['month'] <= end_date)
        return geodata[mask]
    else: return None

In [ ]:
delta = (pd.to_datetime('01/01/2020').to_period('M'), pd.to_datetime('01/01/2022').to_period('M'))
coords(delta)

In [ ]:
@interact(coords, dates=widgets.SelectionRangeSlider(
    index=
    options=
    description='Dates',
    orientation='horizontal',
    layout={'width': '500px'}))

In [ ]:
len(geodata['month'][geodata['month'] > pd.to_datetime('2010-01-02').to_period('M')])

In [ ]:
def on_value_change(change):
    print(change)

selection_range_slider = widgets.SelectionRangeSlider(
    options=geodata['month'][geodata['month'] > pd.to_datetime('2020-01-02').to_period('M')],
    index = (200,1000),
    description='Dates',
    orientation='horizontal',
    layout={'width': '1000px'})

selection_range_slider

In [ ]:
widgets.interact(on_value_change, change=selection_range_slider)

In [ ]:
widgets.interact(coords, period=selection_range_slider)

In [ ]:
location = geodata.dropna(subset=['rad_N'])
location = location[location['month'] == '2023Q2']

1. Сделай селектИнт и поквартально
2. Перевести координаты в градусы

In [ ]:
def render():
    for n in location[['rad_N', 'rad_E']].itertuples(index=False):
        folium.Marker(location=n).add_to(m)

In [ ]:
m = folium.Map(tiles="Stamen Toner", location=location[['rad_N', 'rad_E']].iloc[1], zoom_start=1, prefer_canvas=True, disable_3d=True)
render()
m